In [11]:
#our first version

from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import time

# Setup Firefox WebDriver
# Make sure you have geckodriver installed and available in your PATH
service = Service()
driver = webdriver.Firefox(service=service)

# Open the property page
url = "https://immovlan.be/en/detail/villa/for-sale/9051/sint-denijs-westrem/rbu55821"
driver.get(url)

###########

# ---- Handle cookie consent automatically ----
try:
    # Wait for the "Agree and close" button to be clickable
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
    )
    accept_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("No cookie popup found or could not click the accept button:", e)
    
# ---- End cookie handling ----

#################
time.sleep(10)  # Wait for the page to fully load

# TERRACE: 1 = Yes, 0 = No, None if missing 
try:
    terrace_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Terrace')]/following-sibling::p").text
    terrace = 1 if "Yes" in terrace_text else 0
except:
    terrace = None

# GARDEN: 1 = Yes, 0 = No
try:
    garden_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Garden')]/following-sibling::p").text
    garden = 1 if "Yes" in garden_text else 0
except:
    garden = 0

# GARDEN AREA (m²): integer or None
try:
    garden_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Surface garden')]/following-sibling::p").text
    garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
except:
    garden_area = None

# NUMBER OF FACADES: integer or None 
try:
    facades_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of facades')]/following-sibling::p").text
    facades = int(facades_text)
except:
    facades = None

# SWIMMING POOL: 1 = Yes, 0 = No
try:
    pool_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Swimming pool')]/following-sibling::p").text
    swimming_pool = 1 if "Yes" in pool_text else 0
except:
    swimming_pool = 0

# STATE OF BUILDING: text or None
try:
    state = driver.find_element(By.XPATH, "//h4[contains(text(), 'State of the property')]/following-sibling::p").text
except:
    state = None

# BUILD YEAR: integer or None 
try:
    build_year_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Build Year')]/following-sibling::p").text
    build_year = int(build_year_text)
except:
    build_year = None

# NUMBER OF ROOMS: integer or None
try:
    number_rooms_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of bedrooms')]/following-sibling::p").text
    number_rooms = int(number_rooms_text)
except:
    number_rooms = None

# LIVING AREA (m²): integer or None
try:
    living_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Livable surface')]/following-sibling::p").text
    living_area = int(living_area_text.replace("m²", "").replace("m", "").strip())
except:
    living_area = None

# EQUIPPED KITCHEN: 1 = Yes, 0 = No, None if missing 
# EQUIPPED KITCHEN: 1 = yes, 0 = no, None if missing
try:
    kitchen_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Kitchen equipment')]/following-sibling::p").text
    if "equipped" in kitchen_text.lower():  # any text containing "equipped"
        equipped_kitchen = 1
    elif "not" in kitchen_text.lower():      # text containing "not"
        equipped_kitchen = 0
    else:
        equipped_kitchen = None
except:
    equipped_kitchen = None

# FURNISHED: 1 = Yes, 0 = No, None if missing
try:
    furnished_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Furnished')]/following-sibling::p").text
    if "yes" in furnished_text.lower():
        furnished = 1
    elif "no" in furnished_text.lower():
        furnished = 0
    else:
        furnished = None
except:
    furnished = None

# # OPEN FIRE: 1 = Yes, 0 = No, None if missing 
# try:
#     open_fire_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Open fire')]/following-sibling::p").text
#     if "Yes" in open_fire_text:
#         open_fire = 1
#     elif "No" in open_fire_text:
#         open_fire = 0
#     else:
#         open_fire = None
# except:
#     open_fire = None

# Print results 
print("Terrace:", terrace)
print("Garden presence:", garden)
print("Garden area:", garden_area)
print("Number of facades:", facades)
print("Swimming pool:", swimming_pool)
print("State of building:", state)
print("Build year:", build_year)
print("Number of rooms:", number_rooms)
print("Living area (m²):", living_area)
print("Equipped kitchen:", equipped_kitchen)
print("Furnished:", furnished)
#print("Open fire:", open_fire)

# Close browser 
#driver.quit()

Cookies accepted.
Terrace: 1
Garden presence: 1
Garden area: 470
Number of facades: 4
Swimming pool: 0
State of building: Excellent
Build year: 1954
Number of rooms: 3
Living area (m²): 185
Equipped kitchen: 1
Furnished: 0


# practice code for optimization

In [ ]:

#the cookie based on the advice
# ---- Handle cookie consent automatically ----
try:
    # Wait for the "Agree and close" button to be clickable
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
    )
    accept_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("No cookie popup found or could not click the accept button:", e)
    
# ---- End cookie handling ----

In [3]:

#our working code :-)))))


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait    # added
from selenium.webdriver.support import expected_conditions as EC #added
import csv  # <-- added for saving results


def scrape_properties(urls, output_file="properties.csv"):  # <-- added function definition
    # Firefox options (headless)
    options = Options()
    options.add_argument("--headless")
    options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
 

    # Prepare CSV file
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["url", "property_id", "property_type", "locality_name", "postal_code"", build_year", "state" ,"furnished", "number_rooms","equipped_kitchen", "terrace","facades", "garden", "garden_area","swimming_pool", "price"])

        # Launch Firefox
        GECKO_PATH = r"C:\users\geckodriver.exe"  # <-- your local path
        service = Service(GECKO_PATH)
        driver = webdriver.Firefox(service=service, options=options)
        #print(f"\n Scraping {i} of {len(urls)} → {url}")
        driver.get(url)

        # Loop through URLs
        for url in urls:
            driver = None
            try:
                # Launch Firefox
                GECKO_PATH = r"C:\users\geckodriver.exe"  # <-- your local path
                service = Service(GECKO_PATH)
                driver = webdriver.Firefox(service=service, options=options)
                #print(f"\n Scraping {i} of {len(urls)} → {url}")
                driver.get(url)

                # ---- Handle cookie consent automatically ----
                
                try:
                    # Wait for the "Agree and close" button to be clickable
                    accept_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    accept_button.click()
                    print("Cookies accepted.")
                except Exception as e:
                    print("No cookie popup found or could not click the accept button:", e)
                # ---- End cookie handling ----

                
                ## TERRACE: 1 = Yes, 0 = No
                try:
                    terrace_tag = driver.find_element(By.CSS_SELECTOR, "h4:contains('Terrace') + p")
                    terrace_text = terrace_tag.text.strip()
                    terrace = 1 if "yes" in terrace_text.lower() else 0
                except:
                    terrace = None 

                # GARDEN: 1 = Yes, 0 = No
                try:
                    garden_blocks = driver.find_elements(By.CSS_SELECTOR, "div")
                    garden = 0  # default
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "garden" == h4_text:  # exact match for Garden
                                garden_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden = 1 if "yes" in garden_text.lower() else 0
                                break
                        except:
                            continue
                except:
                    garden = 0

                # GARDEN AREA (m²): integer or None
                try:
                    garden_area = None
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "surface garden" in h4_text:
                                garden_area_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
                                break
                        except:
                            continue
                except:
                    garden_area = None
                try:
                    facades_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of facades')]/following-sibling::p").text
                    facades = int(facades_text)
                except:
                    facades = None


                    ################################  :-)
                # SWIMMING POOL: 1 = Yes, 0 = No
                try:
                    pool_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Swimming pool')]/following-sibling::p").text
                    swimming_pool = 1 if "Yes" in pool_text else 0
                except:
                    swimming_pool = 0


                ###########################"" ADDING MULTIPLE ITESM
                # STATE OF BUILDING: text or None
                try:
                    state = driver.find_element(By.XPATH, "//h4[contains(text(), 'State of the property')]/following-sibling::p").text
                except:
                    state = None

                # BUILD YEAR: integer or None 
                try:
                    build_year_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Build Year')]/following-sibling::p").text
                    build_year = int(build_year_text)
                except:
                    build_year = None

                # NUMBER OF ROOMS: integer or None
                try:
                    number_rooms_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of bedrooms')]/following-sibling::p").text
                    number_rooms = int(number_rooms_text)
                except:
                    number_rooms = None

                    # LIVING AREA (m²): integer or None
                try:
                    living_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Livable surface')]/following-sibling::p").text
                    living_area = int(living_area_text.replace("m²", "").replace("m", "").strip())
                except:
                    living_area = None

                # EQUIPPED KITCHEN: 1 = Yes, 0 = No, None if missing 
            
                try:
                    kitchen_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Kitchen equipment')]/following-sibling::p").text
                    if "equipped" in kitchen_text.lower():  # any text containing "equipped"
                        equipped_kitchen = 1
                    elif "not" in kitchen_text.lower():      # text containing "not"
                        equipped_kitchen = 0
                    else:
                        equipped_kitchen = None
                except:
                    equipped_kitchen = None

                # FURNISHED: 1 = Yes, 0 = No, None if missing
                try:
                    furnished_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Furnished')]/following-sibling::p").text
                    if "yes" in furnished_text.lower():
                        furnished = 1
                    elif "no" in furnished_text.lower():
                        furnished = 0
                    else:
                        furnished = None
                except:
                        furnished = None

                ###########################"" MULTIPLE ITEMS ADDED ABOVE  
                
                

                # -----------------------
                # Extract property ID
                # -----------------------
                try:
                    property_id = url.split('/')[-1]
                except:
                    property_id = None

                # -----------------------
                # Extract postal code & locality
                # -----------------------
                try:
                    city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
                    city_text = city_tag.text.strip()
                    postal_code = city_text.split()[0]
                    locality_name = " ".join(city_text.split()[1:])
                except:
                    postal_code = None
                    locality_name = None

                # -----------------------
                # Extract price
                # -----------------------
                try:
                    price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
                    price = price_tag.text.strip()
                except:
                    price = None

                # -----------------------
                # Extract property type & subtype
                # -----------------------
                try:
                    type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
                    type_words = type_tag.text.strip().split()
                    property_type = type_words[0]
                except:
                    property_type = None

                # -----------------------
                # Print results
                # -----------------------
                print("Terrace:", terrace)
                print("Garden presence:", garden)
                print("Garden area:", garden_area)
                print("Number of facades:", facades)
                print("Swimming pool:", swimming_pool)
                print("State of building:", state)
                print("Build year:", build_year)
                print("Number of rooms:", number_rooms)
                print("Living area (m²):", living_area)
                print("Equipped kitchen:", equipped_kitchen)
                print("Furnished:", furnished)
                print("Property ID:", property_id)
                print("Postal Code:", postal_code)
                print("Locality:", locality_name)
                print("Price:", price)
                print("Type:", property_type)
                

                # Write row to CSV file
                
                writer.writerow([
    url,
    property_id if property_id is not None else None,
    property_type if property_type is not None else None,
    locality_name if locality_name is not None else None,
    postal_code if postal_code is not None else None,
    build_year if build_year is not None else None,
    state if state is not None else None,
    furnished if furnished is not None else None,
    number_rooms if number_rooms is not None else None,
    equipped_kitchen if equipped_kitchen is not None else None,
    terrace if terrace is not None else None,
    facades if facades is not None else None,
    garden if garden is not None else None,
    garden_area if garden_area is not None else None,
    swimming_pool if swimming_pool is not None else None,
    price if price is not None else None,
])
            except Exception as e:
                print(f"Error loading page: {url}")
                print("Reason:", e)
            finally:
                if driver:
                    driver.quit()




# -----------------------------
# ✅ USE SECTION 
# -----------------------------
input_file = "urls-1rst.csv"
output_file = "properties-1rst.csv"

urls = []

# Read URLs from the second column (with header Type,URL)
with open(input_file, mode="r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if "URL" in row and row["URL"].startswith("http"):
            urls.append(row["URL"])

print(f"Loaded {len(urls)} URLs from {input_file}")

# Run the scraper
scrape_properties(urls, output_file)
print(f"✅ Scraping complete. Results saved in '{output_file}'")


Loaded 19 URLs from urls-1rst.csv


UnboundLocalError: cannot access local variable 'url' where it is not associated with a value

In [ ]:
sample result

Loaded 1163 URLs from urls-1rst.csv
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: New
Build year: None
Number of rooms: 2
Living area (m²): 117
Equipped kitchen: None
Furnished: None
Property ID: rbu61969
Postal Code: 3500
Locality: Hasselt
Price: 485 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: 1990
Number of rooms: 1
Living area (m²): 62
Equipped kitchen: None
Furnished: None
Property ID: rbu62018
Postal Code: 3500
Locality: Hasselt
Price: 165 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: 3
Swimming pool: 0
State of building: Excellent
Build year: 2022
Number of rooms: 2
Living area (m²): 110
Equipped kitchen: 1
Furnished: 0
Property ID: rbu61357
Postal Code: 3550
Locality: Heusden-Zolder
Price: 354 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: 4
Swimming pool: 0
State of building: Normal
Build year: 2000
Number of rooms: 2
Living area (m²): 73
Equipped kitchen: 1
Furnished: 0
Property ID: rbu61358
Postal Code: 3550
Locality: Heusden-Zolder
Price: 240 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: 3
Swimming pool: 0
State of building: Excellent
Build year: 2008
Number of rooms: 3
Living area (m²): 160
Equipped kitchen: None
Furnished: 0
Property ID: rbu61359
Postal Code: 3550
Locality: Heusden-Zolder
Price: 319 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 28
Number of facades: None
Swimming pool: 0
State of building: New
Build year: 2024
Number of rooms: 3
Living area (m²): 129
Equipped kitchen: 1
Furnished: None
Property ID: rbu64219
Postal Code: 3550
Locality: Heusden-Zolder
Price: 354 750 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 25
Number of facades: 2
Swimming pool: 0
State of building: New
Build year: 2024
Number of rooms: 2
Living area (m²): 115
Equipped kitchen: 1
Furnished: 0
Property ID: rbu64221
Postal Code: 3550
Locality: Heusden-Zolder
Price: 332 500 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: 2
Swimming pool: 0
State of building: New
Build year: 2021
Number of rooms: 2
Living area (m²): 86
Equipped kitchen: 1
Furnished: None
Property ID: rbu62413
Postal Code: 3580
Locality: Beringen
Price: 329 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: 2024
Number of rooms: 2
Living area (m²): 90
Equipped kitchen: 1
Furnished: 0
Property ID: rbu64215
Postal Code: 3582
Locality: Koersel
Price: 330 500 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 36
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 3
Living area (m²): 137
Equipped kitchen: None
Furnished: None
Property ID: rbu62045
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 342 480 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 34
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 124
Equipped kitchen: None
Furnished: None
Property ID: rbu62046
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 310 560 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 44
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 101
Equipped kitchen: None
Furnished: None
Property ID: rbu62047
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 287 280 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 38
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 88
Equipped kitchen: None
Furnished: None
Property ID: rbu62048
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 261 660 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 33
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 95
Equipped kitchen: None
Furnished: None
Property ID: rbu62049
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 272 700 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: 29
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 95
Equipped kitchen: None
Furnished: None
Property ID: rbu62050
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 272 700 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 104
Equipped kitchen: None
Furnished: None
Property ID: rbu62051
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 284 336 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 99
Equipped kitchen: None
Furnished: None
Property ID: rbu62052
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 274 580 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 116
Equipped kitchen: None
Furnished: None
Property ID: rbu62053
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 311 792 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 86
Equipped kitchen: None
Furnished: None
Property ID: rbu62054
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 246 400 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 91
Equipped kitchen: None
Furnished: None
Property ID: rbu62055
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 257 881 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 89
Equipped kitchen: None
Furnished: None
Property ID: rbu62056
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 258 020 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 2
Living area (m²): 102
Equipped kitchen: None
Furnished: None
Property ID: rbu62057
Postal Code: 3650
Locality: Dilsen-Stokkem
Price: 277 030 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: 2015
Number of rooms: 3
Living area (m²): 151
Equipped kitchen: None
Furnished: None
Property ID: rbu62074
Postal Code: 3680
Locality: Maaseik
Price: 319 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: 1979
Number of rooms: 2
Living area (m²): 110
Equipped kitchen: None
Furnished: None
Property ID: rbu64331
Postal Code: 3680
Locality: Maaseik
Price: 209 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: None
Number of rooms: 3
Living area (m²): 131
Equipped kitchen: None
Furnished: None
Property ID: rbu64524
Postal Code: 3840
Locality: Bommershoven
Price: 239 999 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: Normal
Build year: 2025
Number of rooms: 2
Living area (m²): 104
Equipped kitchen: None
Furnished: None
Property ID: rbu62103
Postal Code: 3900
Locality: Overpelt
Price: 318 500 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: New
Build year: None
Number of rooms: 2
Living area (m²): 99
Equipped kitchen: None
Furnished: None
Property ID: rbu61938
Postal Code: 3900
Locality: Pelt
Price: 318 000 €
Type: Apartment
Cookies accepted.
Terrace: None
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: New
Build year: None
Number of rooms: 2
Living area (m²): 99
Equipped kitchen: None
Furnished: None
Property ID: rbu61939
Postal Code: 3900
Locality: Pelt
Price: 323 000 €
Type: Apartment
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61940
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61941
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61942
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61943
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61944
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61945
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61946
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61947
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61958
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61959
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61960
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61961
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61962
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu61963
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu62101
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3900/pelt/rbu62102
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3910/neerpelt/rbu65861
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3970/leopoldsburg/rbu65082
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/3980/tessenderlo/rbu61984
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd47582
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd47649
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd47667
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd48064
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd48335
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd48597
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd48645
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd48647
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd49310
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4000/liege/vbd49312
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4020/liege/vbd47457
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4020/liege/vbd47848
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4020/liege/vbd48836
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4020/liege/vbd49421
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4020/liege/vbd49584
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4020/liege/vbd49634
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4031/angleur/vbd48822
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4032/chenee/vbd49557
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4053/embourg/vbd44032
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4100/boncelles/vbd47434
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4100/boncelles/vbd48567
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4100/boncelles/vbd48823
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4180/hamoir/vbd47406
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4280/hannut/vbd48403
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4400/flemalle/vbd49049
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4432/xhendremael/vbd49439
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4500/huy/vbd47439
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4500/huy/vbd48451
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4500/huy/vbd49105
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4610/queue-du-bois/vbd47446
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4620/fleron/vbd48290
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4620/fleron/vbd49100
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4728/hergenrath/vbd48205
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4728/hergenrath/vbd48206
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4750/butgenbach/vbd48062
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4780/saint-vith/vbd49116
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4780/saint-vith/vbd49117
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4780/saint-vith/vbd49118
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4780/saint-vith/vbd49119
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4780/saint-vith/vbd49120
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4780/saint-vith/vbd49121
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4780/saint-vith/vbd49122
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4800/ensival/vbd47445
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4870/trooz/vbd49220
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/4870/trooz/vbd49221
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd44892
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd44931
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd45938
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd45972
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd46324
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd46440
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd46949
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd46974
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd47419
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd47646
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vbd48272
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vwd14838
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5000/namur/vwd15077
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5004/bouge/vbd47960
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5004/bouge/vbd49533
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5020/malonne/vbd47450
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5020/malonne/vbd47451
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5020/malonne/vbd47479
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5060/sambreville/vbd43575
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5100/jambes/vbd16271
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5100/jambes/vbd16272
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5100/jambes/vbd16273
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5101/erpent/vbd45442
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5140/ligny/vbd49522
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5150/floreffe/vbd48431
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5170/lesve/vbd46276
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5330/assesse/vbd44665
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5500/dinant/vbd45766
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5500/dinant/vbd48099
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5520/onhaye/vbd45230
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5530/godinne/vbd46610
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5580/rochefort/vbd47943
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5580/rochefort/vbd48486
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5580/rochefort/vbd48491
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5590/ciney/vbd47014
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5590/ciney/vbd47449
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5590/ciney/vbd47477
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/5590/ciney/vbd47864
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6000/charleroi/vbd45714
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6000/charleroi/vbd45718
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6000/charleroi/vbd45770
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6000/charleroi/vbd48936
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6001/marcinelle/vbd49229
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6032/mont-sur-marchienne/vbd48323
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6060/gilly/vbd48821
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6060/gilly/vbd49213
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6140/fontaine-l-eveque/vbd48439
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6200/bouffioulx/vbd48943
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6200/bouffioulx/vbd48944
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6200/chatelet/vbd49214
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/6441/erpion/vbd49329
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7000/mons/vbd48828
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7000/mons/vbd48830
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7000/mons/vbd49562
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7100/la-louviere/vbd47252
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7100/la-louviere/vbd48534
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7100/la-louviere/vbd48535
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7100/la-louviere/vbd48536
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7100/la-louviere/vbd48537
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7100/la-louviere/vwd15387
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7500/tournai/vbd48744
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7500/tournai/vbd49527
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7531/havinnes/vbd48215
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7700/mouscron/vbd48235
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7700/mouscron/vbd48743
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7700/mouscron/vbd48997
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7711/dottignies/vbd49592
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7711/dottignies/vbd49593
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/7800/ath/rbu66042
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8000/brugge/rbu66320
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8210/zedelgem/rbu62613
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8300/knokke-heist/rbu61513
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8300/knokke-heist/rbu66252
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8301/duinbergen/rbu65535
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8310/sint-kruis/rbu65285
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8370/blankenberge/rbu65695
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8370/blankenberge/rbu66130
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8370/blankenberge/rwc41048
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8380/zeebrugge/rwc41045
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8400/oostende/rbu64785
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8400/oostende/rbu65642
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8400/oostende/rbu66233
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8400/oostende/rbu66377
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8430/middelkerke/rbt29590
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8430/middelkerke/rbu55447
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8430/middelkerke/rbu65078
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8450/bredene/rbu64775
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8450/bredene/rbu64788
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8450/bredene/rbu65753
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8500/kortrijk/rbu65031
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8500/kortrijk/rbu65060
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8500/kortrijk/rbu65062
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8500/kortrijk/rbu65974
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8530/harelbeke/rbt69747
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8610/kortemark/rbu65543
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8610/kortemark/rbu65544
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8620/nieuwpoort/rbu65080
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8620/nieuwpoort/rbu66374
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8630/veurne/rbu65221
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8660/de-panne/rbu64928
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8670/koksijde/rbu60582
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8670/koksijde/rbu64793
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8670/koksijde/rbu64794
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8670/koksijde/rbu64795
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8670/koksijde/rbu65273
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8670/koksijde/rbu65293
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8670/koksijde/rbu65294
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8670/koksijde/rbu66373
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8800/roeselare/rbu25477
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8800/roeselare/rbu65283
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8800/roeselare/rbu65284
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8800/roeselare/rbu65689
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8800/roeselare/rbu65690
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8800/roeselare/rbu65691
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8800/roeselare/rbu65692
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/8800/roeselare/rbu65864
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu58960
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu59338
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9000/gent/rbu65487
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9051/sint-denijs-westrem/rbu65231
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9060/zelzate/rbu65228
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9080/zeveneken/rbu66176
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9080/zeveneken/rbu66177
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9080/zeveneken/rbu66179
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9080/zeveneken/rbu66180
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9100/sint-niklaas/rbu65859
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9100/sint-niklaas/rbu65950
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9230/wetteren/rbu65782
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9230/wetteren/rbu65783
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9230/wetteren/rbu65784
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9300/aalst/rbu66325
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9320/erembodegem/rbu66213
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9400/ninove/rbu65956
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9420/erpe-mere/rbu65975
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9450/denderhoutem/rbu66305
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9500/geraardsbergen/rbu65147
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9500/geraardsbergen/rbu65148
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9500/geraardsbergen/rbu65150
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9506/schendelbeke/rbu65149
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9550/herzele/rbu65145
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66344
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66345
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66347
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66348
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66349
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66350
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66351
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66352
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66353
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66354
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66355
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66356
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66357
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9700/oudenaarde/rbu66358
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9800/deinze/rbu66027
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9940/evergem/rbu66312
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9940/evergem/rbu66313
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9940/evergem/rbu66314
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9940/evergem/rbu66315
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9940/evergem/rbu66316
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9940/evergem/rbu66317
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9940/evergem/rbu66318
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9940/evergem/rbu66319
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9950/waarschoot/rbu65536
Error loading page: https://immovlan.be/en/detail/apartment/for-sale/9990/maldegem/rbu65280
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/3530/houthalen-helchteren/rbu62982
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/3550/heusden-zolder/rbu62416
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/3550/heusden-zolder/rbu64220
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/3550/heusden-zolder/rbu65849
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/4100/seraing/vbd48399
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/4140/sprimont/vbd47963
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/5000/namur/vbd47647
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/5100/jambes/vbd47382
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/5150/floreffe/vbd48432
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/5300/andenne/vbd43808
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/5500/anseremme/vbd46612
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/6060/gilly/vbd48798
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/6060/gilly/vbd48799
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/6060/gilly/vbd48800
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/6060/gilly/vbd48801
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/6060/gilly/vbd48802
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/6060/gilly/vbd48803
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/6060/gilly/vbd48804
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/7611/la-glanerie/vbd48303
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/7700/mouscron/vbd48329
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/8000/brugge/rbu64780
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/8620/nieuwpoort/rbu65144
Error loading page: https://immovlan.be/en/detail/duplex/for-sale/8890/moorslede/rbu65714
Error loading page: https://immovlan.be/en/detail/ground-floor/for-sale/3920/lommel/rbu65844
Error loading page: https://immovlan.be/en/detail/ground-floor/for-sale/4031/angleur/vbd48820
Error loading page: https://immovlan.be/en/detail/ground-floor/for-sale/4100/boncelles/vbd48418
Error loading page: https://immovlan.be/en/detail/ground-floor/for-sale/4100/seraing/vbd48398
Error loading page: https://immovlan.be/en/detail/ground-floor/for-sale/5100/jambes/vwd15483
Error loading page: https://immovlan.be/en/detail/ground-floor/for-sale/8310/assebroek/rbu65841
Error loading page: https://immovlan.be/en/detail/ground-floor/for-sale/8530/harelbeke/rbu65070
Error loading page: https://immovlan.be/en/detail/ground-floor/for-sale/9400/denderwindeke/rbu65233
Error loading page: https://immovlan.be/en/detail/loft/for-sale/4000/liege/vbd48887
Error loading page: https://immovlan.be/en/detail/loft/for-sale/8500/kortrijk/rbu65053
Error loading page: https://immovlan.be/en/detail/penthouse/for-sale/3500/hasselt/rbu61408
Error loading page: https://immovlan.be/en/detail/penthouse/for-sale/4500/huy/vbd47438
Error loading page: https://immovlan.be/en/detail/penthouse/for-sale/6032/mont-sur-marchienne/vbd48768
Error loading page: https://immovlan.be/en/detail/penthouse/for-sale/8500/kortrijk/rbu65057
Error loading page: https://immovlan.be/en/detail/penthouse/for-sale/8500/kortrijk/rwc41100
Error loading page: https://immovlan.be/en/detail/penthouse/for-sale/8930/lauwe/rbu65068
Error loading page: https://immovlan.be/en/detail/penthouse/for-sale/9230/wetteren/rbu65785
Error loading page: https://immovlan.be/en/detail/penthouse/for-sale/9900/eeklo/rbu65862
Error loading page: https://immovlan.be/en/detail/studio/for-sale/5000/namur/vbd44626
Error loading page: https://immovlan.be/en/detail/studio/for-sale/5000/namur/vbd46816
Error loading page: https://immovlan.be/en/detail/studio/for-sale/6000/charleroi/vbd44518
Error loading page: https://immovlan.be/en/detail/studio/for-sale/7000/mons/vbd48829
Error loading page: https://immovlan.be/en/detail/studio/for-sale/7500/tournai/vbd48745
Error loading page: https://immovlan.be/en/detail/studio/for-sale/8200/sint-andries/rbu64781
Error loading page: https://immovlan.be/en/detail/studio/for-sale/8200/sint-andries/rbu64782
Error loading page: https://immovlan.be/en/detail/studio/for-sale/8380/zeebrugge/rbu65281
Error loading page: https://immovlan.be/en/detail/studio/for-sale/8400/oostende/rbu64774
Error loading page: https://immovlan.be/en/detail/studio/for-sale/8670/oostduinkerke/rbu14796
Error loading page: https://immovlan.be/en/detail/triplex/for-sale/4500/huy/vbd48967
Error loading page: https://immovlan.be/en/detail/triplex/for-sale/5590/ciney/vbd47330
Error loading page: https://immovlan.be/en/projectdetail/1138760-in00562508_om_18424
Error loading page: https://immovlan.be/en/projectdetail/1138760-in00562508_om_18436
Error loading page: https://immovlan.be/en/projectdetail/1143862-8359159
Error loading page: https://immovlan.be/en/projectdetail/1151029-4182133
Error loading page: https://immovlan.be/en/projectdetail/1152685-7855285
Error loading page: https://immovlan.be/en/projectdetail/1152685-8516151
Error loading page: https://immovlan.be/en/projectdetail/1287076-3614242
Error loading page: https://immovlan.be/en/projectdetail/1290232-1473942
Error loading page: https://immovlan.be/en/projectdetail/1290232-3397087
Error loading page: https://immovlan.be/en/projectdetail/1290232-3701078
Error loading page: https://immovlan.be/en/projectdetail/1304239-vi0001930
Error loading page: https://immovlan.be/en/projectdetail/1304239-vi0002009
Error loading page: https://immovlan.be/en/projectdetail/1305341-7217590
Error loading page: https://immovlan.be/en/projectdetail/1305341-7219259
Error loading page: https://immovlan.be/en/projectdetail/1337813-8510994
Error loading page: https://immovlan.be/en/projectdetail/1337813-8531903
Error loading page: https://immovlan.be/en/projectdetail/1383956-01249060_om_3712
Error loading page: https://immovlan.be/en/projectdetail/1383956-01249060_om_3952
Error loading page: https://immovlan.be/en/projectdetail/1383956-01249060_om_4076
Error loading page: https://immovlan.be/en/projectdetail/1383956-01249060_om_4194
Error loading page: https://immovlan.be/en/projectdetail/1383956-01249060_om_4249
Error loading page: https://immovlan.be/en/projectdetail/1441863-7222871
Error loading page: https://immovlan.be/en/projectdetail/1477687-in00563967_om_33100
Error loading page: https://immovlan.be/en/projectdetail/1477687-in00563967_om_36403
Error loading page: https://immovlan.be/en/projectdetail/1477687-in00563967_om_36517
Error loading page: https://immovlan.be/en/projectdetail/1477687-in00563967_om_36556
Error loading page: https://immovlan.be/en/projectdetail/1506328-3593318
Error loading page: https://immovlan.be/en/projectdetail/1507364-142
Error loading page: https://immovlan.be/en/projectdetail/1507364-146
Error loading page: https://immovlan.be/en/projectdetail/1507364-147
Error loading page: https://immovlan.be/en/projectdetail/1507364-148
Error loading page: https://immovlan.be/en/projectdetail/1507364-9
Error loading page: https://immovlan.be/en/projectdetail/1532470-00933495_om_1024
Error loading page: https://immovlan.be/en/projectdetail/1634773-4160797
Error loading page: https://immovlan.be/en/projectdetail/1637219-7241164
Error loading page: https://immovlan.be/en/projectdetail/1645097-in00001168_om_2580
Error loading page: https://immovlan.be/en/projectdetail/1711099-4182634
Error loading page: https://immovlan.be/en/projectdetail/1711099-4189892
Error loading page: https://immovlan.be/en/projectdetail/1716482-3865686
Error loading page: https://immovlan.be/en/projectdetail/1716482-3883223
Error loading page: https://immovlan.be/en/projectdetail/1716482-3893340
Error loading page: https://immovlan.be/en/projectdetail/1716482-3966239
Error loading page: https://immovlan.be/en/projectdetail/1716482-4029549
Error loading page: https://immovlan.be/en/projectdetail/1716482-4085561
Error loading page: https://immovlan.be/en/projectdetail/1716482-4110772
Error loading page: https://immovlan.be/en/projectdetail/1716482-4190495
Error loading page: https://immovlan.be/en/projectdetail/1756718-3174110
Error loading page: https://immovlan.be/en/projectdetail/1864580-7138737
Error loading page: https://immovlan.be/en/projectdetail/1868427-7202522
Error loading page: https://immovlan.be/en/projectdetail/1876557-8567130
Error loading page: https://immovlan.be/en/projectdetail/1938176-rmg01376437_om_4339
Error loading page: https://immovlan.be/en/projectdetail/1938176-rmg01376437_om_4370
Error loading page: https://immovlan.be/en/projectdetail/1951882-7227267
Error loading page: https://immovlan.be/en/projectdetail/2015299-3432914
Error loading page: https://immovlan.be/en/projectdetail/2015299-3444760
Error loading page: https://immovlan.be/en/projectdetail/2020744-7223311
Error loading page: https://immovlan.be/en/projectdetail/2023606-6079453
Error loading page: https://immovlan.be/en/projectdetail/2023631-5219642
Error loading page: https://immovlan.be/en/projectdetail/2023631-5890860
Error loading page: https://immovlan.be/en/projectdetail/2023631-5942915
Error loading page: https://immovlan.be/en/projectdetail/2023631-6112220
Error loading page: https://immovlan.be/en/projectdetail/2023631-6599198
Error loading page: https://immovlan.be/en/projectdetail/2023631-7079041
Error loading page: https://immovlan.be/en/projectdetail/2023631-7203817
Error loading page: https://immovlan.be/en/projectdetail/2023654-7228136
Error loading page: https://immovlan.be/en/projectdetail/2029639-7241764
Error loading page: https://immovlan.be/en/projectdetail/2108834-7229117
Error loading page: https://immovlan.be/en/projectdetail/2108834-7229180
Error loading page: https://immovlan.be/en/projectdetail/22353-7373968
Error loading page: https://immovlan.be/en/projectdetail/22353-8071115
Error loading page: https://immovlan.be/en/projectdetail/2292533-6642751
Error loading page: https://immovlan.be/en/projectdetail/23459-3602078
Error loading page: https://immovlan.be/en/projectdetail/23459-3699788
Error loading page: https://immovlan.be/en/projectdetail/23459-4054420
Error loading page: https://immovlan.be/en/projectdetail/23459-4155434
Error loading page: https://immovlan.be/en/projectdetail/2365186-038f498b-7f9c-4cf3-bddf-72772c0a8ae2
Error loading page: https://immovlan.be/en/projectdetail/24069-7238040
Error loading page: https://immovlan.be/en/projectdetail/24069-7238307
Error loading page: https://immovlan.be/en/projectdetail/2407531-4060529
Error loading page: https://immovlan.be/en/projectdetail/24106-00053845_om_10346
Error loading page: https://immovlan.be/en/projectdetail/24106-00053845_om_10363
Error loading page: https://immovlan.be/en/projectdetail/24421-3964648
Error loading page: https://immovlan.be/en/projectdetail/24431-p-2275-n2ig4ufr
Error loading page: https://immovlan.be/en/projectdetail/24431-p-2278-nmeu0c5a
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk0101-tfjkrblr
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2184-8at86gmj
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2192-qcoizcun
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2212-scakhae0
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2220-oucrm7ve
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2250-9gucv2na
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2254-fzbjbcd4
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2262-w203rvm2
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2266-bhswwk8e
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2273-dhiftbxl
Error loading page: https://immovlan.be/en/projectdetail/24431-p-vk2293-kfqopjxp
Error loading page: https://immovlan.be/en/projectdetail/24574-4154420
Error loading page: https://immovlan.be/en/projectdetail/24941-6691629
Error loading page: https://immovlan.be/en/projectdetail/24941-6700490
Error loading page: https://immovlan.be/en/projectdetail/24941-6700864
Error loading page: https://immovlan.be/en/projectdetail/24941-6785706
Error loading page: https://immovlan.be/en/projectdetail/24941-6804522
Error loading page: https://immovlan.be/en/projectdetail/24941-6804581
Error loading page: https://immovlan.be/en/projectdetail/24966-2423471
Error loading page: https://immovlan.be/en/projectdetail/2498072-rmg01405617_om_265025
Error loading page: https://immovlan.be/en/projectdetail/2498072-rmg01405617_om_271053
Error loading page: https://immovlan.be/en/projectdetail/2504989-6488136
Error loading page: https://immovlan.be/en/projectdetail/2553831-5537671
Error loading page: https://immovlan.be/en/projectdetail/2553831-7220329
Error loading page: https://immovlan.be/en/projectdetail/25733-3636747
Error loading page: https://immovlan.be/en/projectdetail/25733-3750229
Error loading page: https://immovlan.be/en/projectdetail/25733-3791321
Error loading page: https://immovlan.be/en/projectdetail/25733-4102767
Error loading page: https://immovlan.be/en/projectdetail/25791-4186977
Error loading page: https://immovlan.be/en/projectdetail/25923-7192201
Error loading page: https://immovlan.be/en/projectdetail/25923-7219068
Error loading page: https://immovlan.be/en/projectdetail/25923-7219123
Error loading page: https://immovlan.be/en/projectdetail/25923-7220439
Error loading page: https://immovlan.be/en/projectdetail/25923-7222379
Error loading page: https://immovlan.be/en/projectdetail/25923-7222386
Error loading page: https://immovlan.be/en/projectdetail/25923-7222421
Error loading page: https://immovlan.be/en/projectdetail/25923-7222463
Error loading page: https://immovlan.be/en/projectdetail/25923-7225244
Error loading page: https://immovlan.be/en/projectdetail/25923-7225251
Error loading page: https://immovlan.be/en/projectdetail/25923-7225587
Error loading page: https://immovlan.be/en/projectdetail/25923-7225610
Error loading page: https://immovlan.be/en/projectdetail/25923-7225794
Error loading page: https://immovlan.be/en/projectdetail/25923-7225840
Error loading page: https://immovlan.be/en/projectdetail/25923-7225932
Error loading page: https://immovlan.be/en/projectdetail/25923-7226530
Error loading page: https://immovlan.be/en/projectdetail/25923-7234979
Error loading page: https://immovlan.be/en/projectdetail/25923-7235005
Error loading page: https://immovlan.be/en/projectdetail/25923-7235014
Error loading page: https://immovlan.be/en/projectdetail/25923-7235431
Error loading page: https://immovlan.be/en/projectdetail/25923-7237087
Error loading page: https://immovlan.be/en/projectdetail/25923-7237120
Error loading page: https://immovlan.be/en/projectdetail/25923-7237131
Error loading page: https://immovlan.be/en/projectdetail/25923-7237405
Error loading page: https://immovlan.be/en/projectdetail/25923-7242275
Error loading page: https://immovlan.be/en/projectdetail/25923-7242301
Error loading page: https://immovlan.be/en/projectdetail/25923-7242321
Error loading page: https://immovlan.be/en/projectdetail/2595382-7140891
Error loading page: https://immovlan.be/en/projectdetail/2623660-8783764
Error loading page: https://immovlan.be/en/projectdetail/2623660-8783868
Error loading page: https://immovlan.be/en/projectdetail/2623660-8783934
Error loading page: https://immovlan.be/en/projectdetail/2623664-8783963
Error loading page: https://immovlan.be/en/projectdetail/26817-6665913
Error loading page: https://immovlan.be/en/projectdetail/26817-6665922
Error loading page: https://immovlan.be/en/projectdetail/26817-6684963
Error loading page: https://immovlan.be/en/projectdetail/26850-in00560897_om_7773
Error loading page: https://immovlan.be/en/projectdetail/26850-in00560897_om_8760
Error loading page: https://immovlan.be/en/projectdetail/26855-5410200
Error loading page: https://immovlan.be/en/projectdetail/26855-5866445
Error loading page: https://immovlan.be/en/projectdetail/26855-6294949
Error loading page: https://immovlan.be/en/projectdetail/26855-6340131
Error loading page: https://immovlan.be/en/projectdetail/26855-6346234
Error loading page: https://immovlan.be/en/projectdetail/26931-01061037_om_3928
Error loading page: https://immovlan.be/en/projectdetail/26931-01061037_om_3987
Error loading page: https://immovlan.be/en/projectdetail/26931-01061037_om_3997
Error loading page: https://immovlan.be/en/projectdetail/26931-01061037_om_4006
Error loading page: https://immovlan.be/en/projectdetail/26931-01061037_om_4013
Error loading page: https://immovlan.be/en/projectdetail/26931-01061037_om_4080
Error loading page: https://immovlan.be/en/projectdetail/26931-01061037_om_4095
Error loading page: https://immovlan.be/en/projectdetail/2853502-rmg01422468_om_271976
Error loading page: https://immovlan.be/en/projectdetail/2853502-rmg01422468_om_272721
Error loading page: https://immovlan.be/en/projectdetail/2905433-rmg01424715_om_270842
Error loading page: https://immovlan.be/en/projectdetail/2950353-5529389
Error loading page: https://immovlan.be/en/projectdetail/2950353-5532745
Error loading page: https://immovlan.be/en/projectdetail/2950353-5600787
Error loading page: https://immovlan.be/en/projectdetail/2950353-5822725
Error loading page: https://immovlan.be/en/projectdetail/2950353-6324162
Error loading page: https://immovlan.be/en/projectdetail/3341014-7011316
Error loading page: https://immovlan.be/en/projectdetail/vi0002164
Error loading page: https://immovlan.be/en/projectdetail/vi0002165
Error loading page: https://immovlan.be/en/projectdetail/vi0002166
Error loading page: https://immovlan.be/en/projectdetail/vi0002182
Error loading page: https://immovlan.be/en/projectdetail/vi0002198
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/3941/eksel/rbu65076
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/4460/grace-hollogne/vbd48659
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/4600/vise/vbd47498
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/5540/hastiere-lavaux/vbd46804
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/5660/aublain/vbd45572
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/6441/erpion/vbd48420
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/7190/ecaussinnes/vbd49360
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/8400/oostende/rbu64777
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/8600/diksmuide/rbu65542
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/8930/menen/rwc41129
Error loading page: https://immovlan.be/en/detail/bungalow/for-sale/9600/ronse/rbu66018
Error loading page: https://immovlan.be/en/detail/chalet/for-sale/5377/somme-leuze/vbd46462
Error loading page: https://immovlan.be/en/detail/chalet/for-sale/5503/sorinnes/vbd20265
Error loading page: https://immovlan.be/en/detail/chalet/for-sale/5542/blaimont/rbu65660
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/3640/kinrooi/rbu64303
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/4280/merdorp/vwd15564
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/5000/namur/vbd45095
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/5336/courriere/vbd45094
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/5575/louette-saint-denis/vwd15493
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/8620/nieuwpoort/rbu64865
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/8900/ieper/rbu65802
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/8940/wervik/vbd48530
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/9550/herzele/rbu65940
Error loading page: https://immovlan.be/en/detail/cottage/for-sale/9680/maarkedal/rbu65957
Error loading page: https://immovlan.be/en/detail/mansion/for-sale/3840/borgloon/rbu62802
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3500/hasselt/rbu62069
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3500/hasselt/rbu63164
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3500/hasselt/rbu64389
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3500/hasselt/rbu65851
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3520/zonhoven/rbu62098
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3520/zonhoven/rbu66307
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3530/houthalen-helchteren/rbu65074
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3530/houthalen-helchteren/rbu65967
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3540/schulen/rbu61610
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3550/heusden-zolder/rbu61425
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3550/heusden-zolder/rbu61428
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3550/heusden-zolder/rbu62414
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3550/heusden-zolder/rbu64218
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3550/heusden-zolder/rbu64222
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3550/heusden-zolder/rbu64225
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3550/heusden-zolder/rbu65769
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3550/heusden/rbu64224
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3560/lummen/rbu64413
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3560/lummen/rbu64842
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3560/lummen/rbu65968
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3570/alken/rbu62672
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3580/beringen/rbu62412
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3580/beringen/rbu62562
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3580/beringen/rbu64390
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3580/beringen/rbu64841
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3580/beringen/rbu65259
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3580/beringen/rbu65754
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3580/beringen/rbu65886
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3581/beverlo/rbu64216
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3582/koersel/rbu61427
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3582/koersel/rbu64223
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3582/koersel/rbu64843
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3583/paal/rbu64265
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3583/paal/rbu64471
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3590/diepenbeek/rbu62094
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3590/diepenbeek/rbu65791
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3600/genk/rbu62022
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3600/genk/rbu62036
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3600/genk/rbu64243
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3600/genk/rbu65850
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3621/rekem/rbu62497
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3630/maasmechelen/rbu61607
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3630/maasmechelen/rbu64305
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3630/maasmechelen/rbu64922
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3630/maasmechelen/rbu64951
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3630/maasmechelen/rbu65770
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3630/maasmechelen/rbu65883
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3631/boorsem/rbu62494
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3640/kinrooi/rbu61397
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3640/kinrooi/rbu62058
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3640/kinrooi/rbu63265
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3640/kinrooi/rbu64304
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3650/dilsen-stokkem/rbu61606
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3650/dilsen-stokkem/rbu62380
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3650/dilsen-stokkem/rbu62973
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3650/dilsen-stokkem/rbu64840
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3650/dilsen-stokkem/rwc41140
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3650/dilsen-stokkem/rwc41141
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3665/as/rbu61411
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3680/maaseik/rbu62972
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3680/neeroeteren/rbu64241
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3690/zutendaal/rbu62983
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3700/berg/rbu61983
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3700/berg/rbu62657
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3700/berg/rbu62658
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3700/berg/rbu62659
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3700/berg/rbu62671
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3700/berg/rbu65083
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3700/tongeren/rbu64844
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3720/kortessem/rbu63163
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3740/beverst/rbu65275
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3790/sint-martens-voeren/vbd46658
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3791/remersdaal/vbd49505
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3800/sint-truiden/rbu62000
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3800/sint-truiden/rbu65765
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3800/sint-truiden/rbu65932
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3800/sint-truiden/rbu65955
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3800/sint-truiden/rbu66053
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3830/wellen/rbu62848
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3830/wellen/rbu65637
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3840/bommershoven/rbu61426
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3840/borgloon/rbu62092
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3850/nieuwerkerken/rbu62061
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3870/heers/rbu65931
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3890/gingelom/rbu62096
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3890/montenaken/rbu62720
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/overpelt/rbu64267
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/overpelt/rbu65842
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/overpelt/rbu65854
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61948
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61949
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61950
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61951
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61952
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61953
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61954
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61955
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61956
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61957
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61964
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61965
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61966
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61967
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu61968
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3900/pelt/rbu64268
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3920/lommel/rbu62709
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3920/lommel/rbu65081
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3920/lommel/rbu65771
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3920/lommel/rbu65949
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3930/hamont-achel/rbu62154
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3930/hamont-achel/rbu65772
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3940/hechtel/rbu62038
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3945/ham/rbu62093
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3945/ham/rbu62383
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3945/ham/rbu62652
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3945/ham/rbu64904
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3970/leopoldsburg/rbu62649
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3970/leopoldsburg/rbu64335
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3970/leopoldsburg/rbu65260
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3970/leopoldsburg/rbu65314
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3980/tessenderlo/rbu64217
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3980/tessenderlo/rbu64411
Error loading page: https://immovlan.be/en/detail/residence/for-sale/3990/peer/rbu62352
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4000/glain/vbd48648
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4000/liege/vbd47403
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4000/liege/vbd47704
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4000/liege/vbd47897
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4000/liege/vbd48644
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4000/liege/vbd48809
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4000/liege/vbd49249
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4000/liege/vbd49542
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4020/bressoux/vbd48897
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4020/liege/vbd48834
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4030/grivegnee/vbd32524
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4030/grivegnee/vbd48150
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4030/grivegnee/vbd49316
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4040/herstal/rbu64337
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4040/herstal/vbd48291
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4040/herstal/vbd48330
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4040/herstal/vbd49355
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4041/milmort/vbd48835
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4041/milmort/vbd49264
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4041/vottem/vbd48896
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4051/vaux-sous-chevremont/vbd47484
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4051/vaux-sous-chevremont/vbd48895
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4100/seraing/vbd47932
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4100/seraing/vbd48462
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4100/seraing/vbd48539
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4100/seraing/vbd48842
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4100/seraing/vbd49101
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4102/ougree/vbd48060
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4102/ougree/vbd48179
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4121/neuville-en-condroz/vbd48945
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4121/neuville-en-condroz/vbd48946
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4130/tilff/vbd47404
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4130/tilff/vbd48154
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4140/sprimont/vbd48478
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4141/louveigne/vbd48477
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4141/louveigne/vbd49540
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4171/poulseur/vbd47633
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4180/comblain-fairon/vbd47806
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4190/ferrieres/vbd48332
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4190/xhoris/vbd45833
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4217/heron/vbd47983
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4219/wasseiges/vbd44721
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4260/ciplet/vwd15585
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4287/lincent/vbd48813
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4300/bettincourt/vbd47851
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4300/waremme/vbd47956
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4300/waremme/vbd49267
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4340/awans/vbd48516
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4347/fexhe-le-haut-clocher/vbd49353
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4357/donceel/vbd49248
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4360/oreye/vbd48815
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4360/oreye/vbd48816
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4400/flemalle/vbd48558
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4400/flemalle/vbd48833
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4400/flemalle/vbd49232
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4400/flemalle/vbd49246
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4400/flemalle/vbd49247
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4400/flemalle/vbd49559
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4400/ivoz-ramet/vbd48551
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4420/saint-nicolas/vbd48346
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4420/saint-nicolas/vbd49044
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4420/saint-nicolas/vbd49200
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4420/saint-nicolas/vbd49544
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4430/ans/vbd49103
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4430/ans/vbd49245
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4430/ans/vbd49351
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4432/alleur/vbd49032
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4432/alleur/vbd49651
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4460/grace-hollogne/vbd48339
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4470/saint-georges-sur-meuse/vbd48397
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4470/saint-georges-sur-meuse/vbd48864
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4480/engis/vbd48527
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4500/huy/vbd47876
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4500/huy/vbd47895
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4540/amay/vbd48357
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4560/clavier/vbd49569
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4577/modave/vbd47936
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4577/modave/vbd48814
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4577/modave/vbd48950
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4577/modave/vbd48951
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4610/beyne-heusay/vbd49328
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4620/fleron/vbd48893
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4651/battice/vbd47442
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4654/charneux/vbd48075
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4654/charneux/vbd48076
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4671/housse/vbd49438
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4680/oupeye/vbd47496
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4682/heure-le-romain/vbd48947
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4682/heure-le-romain/vbd48948
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4682/houtain-saint-simeon/vbd49074
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4683/vivegnis/vbd48463
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4683/vivegnis/vbd48894
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4683/vivegnis/vbd49614
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4684/haccourt/vbd48077
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4690/glons/vbd48891
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4690/glons/vbd49354
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4690/wonck/vbd48892
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4700/eupen/vbd48777
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4760/bullingen/vbd48540
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4780/recht/vwd15127
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4790/burg-reuland/vbd48377
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4800/petit-rechain/vbd47448
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4800/verviers/vbd48665
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4800/verviers/vbd49308
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4800/verviers/vbd49309
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4800/verviers/vbd49546
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4802/heusy/vbd49028
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4840/welkenraedt/vbd47516
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4880/aubel/vbd49504
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4900/spa/vbd48417
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4950/waimes/vbd47914
Error loading page: https://immovlan.be/en/detail/residence/for-sale/4987/la-gleize/vbd49191
Error loading page: https://immovlan.be/en/detail/residence/for-sale/5000/namur/vbd46562

# our working code a copy


In [3]:



from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait    # added
from selenium.webdriver.support import expected_conditions as EC #added
import csv  # <-- added for saving results


def scrape_properties(urls, output_file="properties.csv"):  # <-- added function definition
    # Firefox options (headless)
    options = Options()
    options.add_argument("--headless")
    options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
 

    # Prepare CSV file
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["URL", "Terrace", "Garden", "Garden area (m²)", "Property ID", "Postal Code", "Locality", "Price", "Type"])

        # Loop through URLs
        for url in urls:
            driver = None
            try:
                # Launch Firefox
                GECKO_PATH = r"C:\users\geckodriver.exe"  # <-- your local path
                service = Service(GECKO_PATH)
                driver = webdriver.Firefox(service=service, options=options)
                #print(f"\n Scraping {i} of {len(urls)} → {url}")
                driver.get(url)
                print(url)

                # ---- Handle cookie consent automatically ----
                
                try:
                    # Wait for the "Agree and close" button to be clickable
                    accept_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    accept_button.click()
                    print("Cookies accepted.")
                except Exception as e:
                    print("No cookie popup found or could not click the accept button:", e)
                # ---- End cookie handling ----

                
                # TERRACE: 1 = Yes, 0 = No, None if missing 
                try:
                    terrace_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Terrace')]/following-sibling::p").text
                    terrace = 1 if "Yes" in terrace_text else 0
                except:
                    terrace = None

                # GARDEN: 1 = Yes, 0 = No
                try:
                    garden_blocks = driver.find_elements(By.CSS_SELECTOR, "div")
                    garden = 0  # default
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "garden" == h4_text:  # exact match for Garden
                                garden_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden = 1 if "yes" in garden_text.lower() else 0
                                break
                        except:
                            continue
                except:
                    garden = 0

                # GARDEN AREA (m²): integer or None
                try:
                    garden_area = None
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "surface garden" in h4_text:
                                garden_area_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
                                break
                        except:
                            continue
                except:
                    garden_area = None
                try:
                    facades_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of facades')]/following-sibling::p").text
                    facades = int(facades_text)
                except:
                    facades = None


                    ################################  :-)
                # SWIMMING POOL: 1 = Yes, 0 = No
                try:
                    pool_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Swimming pool')]/following-sibling::p").text
                    swimming_pool = 1 if "Yes" in pool_text else 0
                except:
                    swimming_pool = 0


                ###########################"" ADDING MULTIPLE ITESM
                # STATE OF BUILDING: text or None
                try:
                    state = driver.find_element(By.XPATH, "//h4[contains(text(), 'State of the property')]/following-sibling::p").text
                except:
                    state = None

                # BUILD YEAR: integer or None 
                try:
                    build_year_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Build Year')]/following-sibling::p").text
                    build_year = int(build_year_text)
                except:
                    build_year = None

                # NUMBER OF ROOMS: integer or None
                try:
                    number_rooms_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of bedrooms')]/following-sibling::p").text
                    number_rooms = int(number_rooms_text)
                except:
                    number_rooms = None

                    # LIVING AREA (m²): integer or None
                try:
                    living_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Livable surface')]/following-sibling::p").text
                    living_area = int(living_area_text.replace("m²", "").replace("m", "").strip())
                except:
                    living_area = None

                # EQUIPPED KITCHEN: 1 = Yes, 0 = No, None if missing 
            
                try:
                    kitchen_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Kitchen equipment')]/following-sibling::p").text
                    if "equipped" in kitchen_text.lower():  # any text containing "equipped"
                        equipped_kitchen = 1
                    elif "not" in kitchen_text.lower():      # text containing "not"
                        equipped_kitchen = 0
                    else:
                        equipped_kitchen = None
                except:
                    equipped_kitchen = None

                # FURNISHED: 1 = Yes, 0 = No, None if missing
                try:
                    furnished_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Furnished')]/following-sibling::p").text
                    if "yes" in furnished_text.lower():
                        furnished = 1
                    elif "no" in furnished_text.lower():
                        furnished = 0
                    else:
                        furnished = None
                except:
                        furnished = None

                ###########################"" MULTIPLE ITEMS ADDED ABOVE  
                
                

                # -----------------------
                # Extract property ID
                # -----------------------
                try:
                    property_id = url.split('/')[-1]
                except:
                    property_id = None

                # -----------------------
                # Extract postal code & locality
                # -----------------------
                try:
                    city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
                    city_text = city_tag.text.strip()
                    postal_code = city_text.split()[0]
                    locality_name = " ".join(city_text.split()[1:])
                except:
                    postal_code = None
                    locality_name = None

                # -----------------------
                # Extract price
                # -----------------------
                try:
                    price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
                    price = price_tag.text.strip()
                except:
                    price = None

                # -----------------------
                # Extract property type & subtype
                # -----------------------
                try:
                    type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
                    type_words = type_tag.text.strip().split()
                    property_type = type_words[0]
                except:
                    property_type = None

                # -----------------------
                # Print results
                # -----------------------
                print("Terrace:", terrace)
                print("Garden presence:", garden)
                print("Garden area:", garden_area)
                print("Number of facades:", facades)
                print("Swimming pool:", swimming_pool)
                print("State of building:", state)
                print("Build year:", build_year)
                print("Number of rooms:", number_rooms)
                print("Living area (m²):", living_area)
                print("Equipped kitchen:", equipped_kitchen)
                print("Furnished:", furnished)
                print("Property ID:", property_id)
                print("Postal Code:", postal_code)
                print("Locality:", locality_name)
                print("Price:", price)
                print("Type:", property_type)
                

                # Write row to CSV file
                
                writer.writerow([url, property_id, property_type, locality_name, postal_code, build_year, state ,furnished, number_rooms,equipped_kitchen, terrace,facades, garden, garden_area,swimming_pool, price,])
            except Exception as e:
                print(f"Error loading page: {url}")
                print("Reason:", e)
            finally:
                if driver:
                    driver.quit()




# -----------------------------
# ✅ USE SECTION 
# -----------------------------
input_file = "urls-1rst.csv"
output_file = "properties-1rst.csv"

urls = []

# Read URLs from the second column (with header Type,URL)
with open(input_file, mode="r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if "URL" in row and row["URL"].startswith("http"):
            urls.append(row["URL"])

print(f"Loaded {len(urls)} URLs from {input_file}")

# Run the scraper
scrape_properties(urls, output_file)
print(f"✅ Scraping complete. Results saved in '{output_file}'")



Loaded 9 URLs from urls-1rst.csv
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbu61969
Cookies accepted.
Terrace: 1
Garden presence: 1
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: New
Build year: None
Number of rooms: 2
Living area (m²): 117
Equipped kitchen: None
Furnished: None
Property ID: rbu61969
Postal Code: 3500
Locality: Hasselt
Price: 485 000 €
Type: Apartment
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbu62018
Cookies accepted.
Terrace: 1
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: 1990
Number of rooms: 1
Living area (m²): 62
Equipped kitchen: None
Furnished: None
Property ID: rbu62018
Postal Code: 3500
Locality: Hasselt
Price: 165 000 €
Type: Apartment
https://immovlan.be/en/detail/apartment/for-sale/3550/heusden-zolder/rbu61357
Cookies accepted.
Terrace: 1
Garden presence: 0
Garden area: None
Number of facades: 3
Swimming pool: 0
State o

In [ ]:



from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait    # added
from selenium.webdriver.support import expected_conditions as EC #added
import csv  # <-- added for saving results


def scrape_properties(urls, output_file="properties.csv"):  # <-- added function definition
    # Firefox options (headless)
    options = Options()
    options.add_argument("--headless")
    options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
 

    # Prepare CSV file
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["url", "property_id", "property_type", "locality_name", "postal_code", "build_year", "state" ,"furnished", "number_rooms","equipped_kitchen", "terrace,facades", "garden", "garden_area","swimming_pool", "price"])

        # Loop through URLs
        for url in urls:
            print(url)
            driver = None
            try:
                # Launch Firefox
                GECKO_PATH = r"C:\users\geckodriver.exe"  # <-- your local path
                service = Service(GECKO_PATH)
                driver = webdriver.Firefox(service=service, options=options)
                #print(f"\n Scraping {i} of {len(urls)} → {url}")
                driver.get(url)

                # ---- Handle cookie consent automatically ----
                
                try:
                    # Wait for the "Agree and close" button to be clickable
                    accept_button = WebDriverWait(driver, 15).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    accept_button.click()
                    print("Cookies accepted.")
                except Exception as e:
                    print("No cookie popup found or could not click the accept button:", e)
                # ---- End cookie handling ----

                
                ## TERRACE: 1 = Yes, 0 = No
                try:
                    terrace_tag = driver.find_element(By.CSS_SELECTOR, "h4:contains('Terrace') + p")
                    terrace_text = terrace_tag.text.strip()
                    terrace = 1 if "yes" in terrace_text.lower() else 0
                except:
                    terrace = None 

                # GARDEN: 1 = Yes, 0 = No
                try:
                    garden_blocks = driver.find_elements(By.CSS_SELECTOR, "div")
                    garden = 0  # default
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "garden" == h4_text:  # exact match for Garden
                                garden_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden = 1 if "yes" in garden_text.lower() else 0
                                break
                        except:
                            continue
                except:
                    garden = 0

                # GARDEN AREA (m²): integer or None
                try:
                    garden_area = None
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "surface garden" in h4_text:
                                garden_area_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
                                break
                        except:
                            continue
                except:
                    garden_area = None
                try:
                    facades_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of facades')]/following-sibling::p").text
                    facades = int(facades_text)
                except:
                    facades = None


                    ################################  :-)
                # SWIMMING POOL: 1 = Yes, 0 = No
                try:
                    pool_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Swimming pool')]/following-sibling::p").text
                    swimming_pool = 1 if "Yes" in pool_text else 0
                except:
                    swimming_pool = 0


                ###########################"" ADDING MULTIPLE ITESM
                # STATE OF BUILDING: text or None
                try:
                    state = driver.find_element(By.XPATH, "//h4[contains(text(), 'State of the property')]/following-sibling::p").text
                except:
                    state = None

                # BUILD YEAR: integer or None 
                try:
                    build_year_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Build Year')]/following-sibling::p").text
                    build_year = int(build_year_text)
                except:
                    build_year = None

                # NUMBER OF ROOMS: integer or None
                try:
                    number_rooms_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of bedrooms')]/following-sibling::p").text
                    number_rooms = int(number_rooms_text)
                except:
                    number_rooms = None

                    # LIVING AREA (m²): integer or None
                try:
                    living_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Livable surface')]/following-sibling::p").text
                    living_area = int(living_area_text.replace("m²", "").replace("m", "").strip())
                except:
                    living_area = None

                # EQUIPPED KITCHEN: 1 = Yes, 0 = No, None if missing 
            
                try:
                    kitchen_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Kitchen equipment')]/following-sibling::p").text
                    if "equipped" in kitchen_text.lower():  # any text containing "equipped"
                        equipped_kitchen = 1
                    elif "not" in kitchen_text.lower():      # text containing "not"
                        equipped_kitchen = 0
                    else:
                        equipped_kitchen = None
                except:
                    equipped_kitchen = None

                # FURNISHED: 1 = Yes, 0 = No, None if missing
                try:
                    furnished_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Furnished')]/following-sibling::p").text
                    if "yes" in furnished_text.lower():
                        furnished = 1
                    elif "no" in furnished_text.lower():
                        furnished = 0
                    else:
                        furnished = None
                except:
                        furnished = None

                ###########################"" MULTIPLE ITEMS ADDED ABOVE  
                
                

                # -----------------------
                # Extract property ID
                # -----------------------
                try:
                    property_id = url.split('/')[-1]
                except:
                    property_id = None

                # -----------------------
                # Extract postal code & locality
                # -----------------------
                try:
                    city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
                    city_text = city_tag.text.strip()
                    postal_code = city_text.split()[0]
                    locality_name = " ".join(city_text.split()[1:])
                except:
                    postal_code = None
                    locality_name = None

                # -----------------------
                # Extract price
                # -----------------------
                try:
                    price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
                    price = price_tag.text.strip()
                except:
                    price = None

                # -----------------------
                # Extract property type & subtype
                # -----------------------
                try:
                    type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
                    type_words = type_tag.text.strip().split()
                    property_type = type_words[0]
                except:
                    property_type = None

                # -----------------------
                # Print results
                # -----------------------
                print("Terrace:", terrace)
                print("Garden presence:", garden)
                print("Garden area:", garden_area)
                print("Number of facades:", facades)
                print("Swimming pool:", swimming_pool)
                print("State of building:", state)
                print("Build year:", build_year)
                print("Number of rooms:", number_rooms)
                print("Living area (m²):", living_area)
                print("Equipped kitchen:", equipped_kitchen)
                print("Furnished:", furnished)
                print("Property ID:", property_id)
                print("Postal Code:", postal_code)
                print("Locality:", locality_name)
                print("Price:", price)
                print("Type:", property_type)
                

                # Write row to CSV file
                
                row = [
                        url, property_id, property_type, locality_name, postal_code,
                        build_year, state, furnished, number_rooms, equipped_kitchen,
                        terrace, facades, garden, garden_area, swimming_pool, price
                    ]

                # Replace empty strings or None with literal "None"
                cleaned_row = [item if (item not in [None, ""]) else "None" for item in row]

                writer.writerow(cleaned_row)
            except Exception as e:
                print(f"Error loading page: {url}")
                print("Reason:", e)
            finally:
                if driver:
                    driver.quit()




# -----------------------------
# ✅ USE SECTION 
# -----------------------------
input_file = "urls_part_1.csv"
output_file = "properties-1rst.csv"

urls = []

# Read URLs from the second column (with header Type,URL)
with open(input_file, mode="r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if "URL" in row and row["URL"].startswith("http"):
            urls.append(row["URL"])

print(f"Loaded {len(urls)} URLs from {input_file}")

# Run the scraper
scrape_properties(urls, output_file)
print(f"✅ Scraping complete. Results saved in '{output_file}'")



Loaded 1000 URLs from urls_part_1.csv
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbs97482
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: New
Build year: 2017
Number of rooms: 1
Living area (m²): 60
Equipped kitchen: None
Furnished: 0
Property ID: rbs97482
Postal Code: 3500
Locality: Hasselt
Price: 189 000 €
Type: Apartment
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbu17157
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
State of building: None
Build year: 1979
Number of rooms: 2
Living area (m²): 90
Equipped kitchen: None
Furnished: 0
Property ID: rbu17157
Postal Code: 3500
Locality: Hasselt
Price: 197 000 €
Type: Apartment
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbu24804
Cookies accepted.
Terrace: None
Garden presence: 0
Garden area: None
Number of facades: None
Swimming pool: 0
Stat

#copy of our working code + try for replacing None in the final dictionary

In [4]:



from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait    # added
from selenium.webdriver.support import expected_conditions as EC #added
import csv  # <-- added for saving results


def scrape_properties(urls, output_file="properties.csv"):  # <-- added function definition
    # Firefox options (headless)
    options = Options()
    options.add_argument("--headless")
    options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
 

    # Prepare CSV file
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["url", "property_id", "property_type", "locality_name", "postal_code",
                        "build_year", "state", "living_area","furnished", "number_rooms", "equipped_kitchen",
                        "terrace", "facades", "garden", "garden_area", "swimming_pool", "price"])

        # Loop through URLs
        for url in urls:
            driver = None
            try:
                # Launch Firefox
                GECKO_PATH = r"C:\users\geckodriver.exe"  # <-- your local path
                service = Service(GECKO_PATH)
                driver = webdriver.Firefox(service=service, options=options)
                #print(f"\n Scraping {i} of {len(urls)} → {url}")
                driver.get(url)
                print(url)

                # ---- Handle cookie consent automatically ----
                
                try:
                    # Wait for the "Agree and close" button to be clickable
                    accept_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
                    )
                    accept_button.click()
                    print("Cookies accepted.")
                except Exception as e:
                    print("No cookie popup found or could not click the accept button:", e)
                # ---- End cookie handling ----

                
                # TERRACE: 1 = Yes, 0 = No, None if missing 
                try:
                    terrace_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Terrace')]/following-sibling::p").text
                    terrace = 1 if "Yes" in terrace_text else 0
                except:
                    terrace = None

                # GARDEN: 1 = Yes, 0 = No
                try:
                    garden_blocks = driver.find_elements(By.CSS_SELECTOR, "div")
                    garden = 0  # default
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "garden" == h4_text:  # exact match for Garden
                                garden_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden = 1 if "yes" in garden_text.lower() else 0
                                break
                        except:
                            continue
                except:
                    garden = 0

                # GARDEN AREA (m²): integer or None
                try:
                    garden_area = None
                    for block in garden_blocks:
                        try:
                            h4_text = block.find_element(By.CSS_SELECTOR, "h4").text.strip().lower()
                            if "surface garden" in h4_text:
                                garden_area_text = block.find_element(By.CSS_SELECTOR, "p").text.strip()
                                garden_area = int(garden_area_text.replace("m²", "").replace("m", "").strip())
                                break
                        except:
                            continue
                except:
                    garden_area = None
                try:
                    facades_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of facades')]/following-sibling::p").text
                    facades = int(facades_text)
                except:
                    facades = None


                    ################################  :-)
                # SWIMMING POOL: 1 = Yes, 0 = No
                try:
                    pool_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Swimming pool')]/following-sibling::p").text
                    swimming_pool = 1 if "Yes" in pool_text else 0
                except:
                    swimming_pool = 0


                ###########################"" ADDING MULTIPLE ITESM
                # STATE OF BUILDING: text or None
                try:
                    state = driver.find_element(By.XPATH, "//h4[contains(text(), 'State of the property')]/following-sibling::p").text
                except:
                    state = None

                # BUILD YEAR: integer or None 
                try:
                    build_year_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Build Year')]/following-sibling::p").text
                    build_year = int(build_year_text)
                except:
                    build_year = None

                # NUMBER OF ROOMS: integer or None
                try:
                    number_rooms_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Number of bedrooms')]/following-sibling::p").text
                    number_rooms = int(number_rooms_text)
                except:
                    number_rooms = None

                    # LIVING AREA (m²): integer or None
                try:
                    living_area_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Livable surface')]/following-sibling::p").text
                    living_area = int(living_area_text.replace("m²", "").replace("m", "").strip())
                except:
                    living_area = None

                # EQUIPPED KITCHEN: 1 = Yes, 0 = No, None if missing 
            
                try:
                    kitchen_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Kitchen equipment')]/following-sibling::p").text
                    if "equipped" in kitchen_text.lower():  # any text containing "equipped"
                        equipped_kitchen = 1
                    elif "not" in kitchen_text.lower():      # text containing "not"
                        equipped_kitchen = 0
                    else:
                        equipped_kitchen = None
                except:
                    equipped_kitchen = None

                # FURNISHED: 1 = Yes, 0 = No, None if missing
                try:
                    furnished_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Furnished')]/following-sibling::p").text
                    if "yes" in furnished_text.lower():
                        furnished = 1
                    elif "no" in furnished_text.lower():
                        furnished = 0
                    else:
                        furnished = None
                except:
                        furnished = None

                ###########################"" MULTIPLE ITEMS ADDED ABOVE  
                
                

                # -----------------------
                # Extract property ID
                # -----------------------
                try:
                    property_id = url.split('/')[-1]
                except:
                    property_id = None

                # -----------------------
                # Extract postal code & locality
                # -----------------------
                try:
                    city_tag = driver.find_element(By.CSS_SELECTOR, ".city-line")
                    city_text = city_tag.text.strip()
                    postal_code = city_text.split()[0]
                    locality_name = " ".join(city_text.split()[1:])
                except:
                    postal_code = None
                    locality_name = None

                # -----------------------
                # Extract price
                # -----------------------
                try:
                    price_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_price_data")
                    price = price_tag.text.strip()
                except:
                    price = None

                # -----------------------
                # Extract property type & subtype
                # -----------------------
                try:
                    type_tag = driver.find_element(By.CSS_SELECTOR, ".detail__header_title_main")
                    type_words = type_tag.text.strip().split()
                    property_type = type_words[0]
                except:
                    property_type = None

                # -----------------------
                # Print results
                # -----------------------
                print("Property ID:", property_id)
                print("Type:", property_type)
                print("Locality:", locality_name)
                print("Postal Code:", postal_code)
                print("Build year:", build_year)
                print("State of building:", state)
                print("Living area (m²):", living_area)
                print("Furnished:", furnished)
                print("Number of rooms:", number_rooms)
                print("Equipped kitchen:", equipped_kitchen)
                print("Terrace:", terrace)
                print("Number of facades:", facades)
                print("Garden presence:", garden)
                print("Garden area:", garden_area)
                print("Swimming pool:", swimming_pool)
                print("Price:", price)
                
                
                
                
                
                
                
               
                
            
                
                

                ## Write row to CSV file
                
                row = [
                        url, property_id, property_type, locality_name, postal_code,
                        build_year, state, furnished, number_rooms, equipped_kitchen,
                        terrace, facades, garden, garden_area, swimming_pool, price
                    ]

                # Replace empty strings or None with literal "None"
                cleaned_row = [item if (item not in [None, ""]) else "None" for item in row]

                writer.writerow(cleaned_row)
            except Exception as e:
                print(f"Error loading page: {url}")
                print("Reason:", e)
            finally:
                if driver:
                    driver.quit()




# -----------------------------
# ✅ USE SECTION 
# -----------------------------
input_file = "urls-1rst.csv"
output_file = "properties-1rst.csv"

urls = []

# Read URLs from the second column (with header Type,URL)
with open(input_file, mode="r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if "URL" in row and row["URL"].startswith("http"):
            urls.append(row["URL"])

print(f"Loaded {len(urls)} URLs from {input_file}")

# Run the scraper
scrape_properties(urls, output_file)
print(f"✅ Scraping complete. Results saved in '{output_file}'")



Loaded 9 URLs from urls-1rst.csv
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbu61969
Cookies accepted.
Property ID: rbu61969
Type: Apartment
Locality: Hasselt
Postal Code: 3500
Build year: None
State of building: New
Living area (m²): 117
Furnished: None
Number of rooms: 2
Equipped kitchen: None
Terrace: 1
Number of facades: None
Garden presence: 1
Garden area: None
Swimming pool: 0
Price: 485 000 €
https://immovlan.be/en/detail/apartment/for-sale/3500/hasselt/rbu62018
Cookies accepted.
Property ID: rbu62018
Type: Apartment
Locality: Hasselt
Postal Code: 3500
Build year: 1990
State of building: None
Living area (m²): 62
Furnished: None
Number of rooms: 1
Equipped kitchen: None
Terrace: 1
Number of facades: None
Garden presence: 0
Garden area: None
Swimming pool: 0
Price: 165 000 €
https://immovlan.be/en/detail/apartment/for-sale/3550/heusden-zolder/rbu61357
Cookies accepted.
Property ID: rbu61357
Type: Apartment
Locality: Heusden-Zolder
Postal Code: 3550
Build year: 